# 基于TF使用NotMnist数据集训练一个线性模型

In [8]:
import pickle
import numpy as np
import tensorflow as tf
import os
import time

image_size = 28
num_labels = 10

# 封装数据集为对象

In [9]:
class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        class Valid:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()
        self.valid = Valid()
        
        pickle_file = os.getcwd() + '/../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.valid.images = valid_dataset
        self.valid.labels = valid_labels
        self.test.images = test_dataset
        self.test.labels = test_labels

# 创建数据集实例

In [10]:
not_mnist = NotMNIST()

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


# 训练参数设定

In [11]:
# 参数
learning_rate = 0.001  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 4  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))


In [12]:
# 构造模型
pred = tf.nn.softmax(tf.matmul(x, W) + b)  # Softmax

In [13]:
# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# 使用梯度下降算法寻找最优解
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# 测试模型
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# 计算准确度
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 初始化所有变量
init = tf.initialize_all_variables()
saver = tf.train.Saver()

# 开始进行训练，按照迭代次数进行迭代，每个迭代采用valid数据集进行验证

In [7]:
# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量
    t0 = time.time()
    # 迭代训练
    for epoch in range(training_epochs):
        t1 = time.time()
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost), "valid=", accuracy.eval({x: not_mnist.valid.images, y: not_mnist.valid.labels}))
            print("Epoch:", '%04d' % (epoch + 1), ' epoch time:', time.time() - t1)
    print('fit time:', time.time() - t0)
    print("Optimization Finished!")
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    #saver.save(sess, 'my-model') # 保存模型

Epoch: 0001 cost= 0.675871590 valid= 0.8283
Epoch: 0002 cost= 0.639088612 valid= 0.8302
Epoch: 0003 cost= 0.633627424 valid= 0.8312
Epoch: 0004 cost= 0.630671003 valid= 0.8314
Optimization Finished!
Accuracy: 0.898


# 作业一：请尝试切换不同的优化方法，例如使用：
* GradientDescentOptimizer
* AdadeltaOptimizer
* AdagradOptimizer
* MomentumOptimizer
* AdamOptimizer
* FtrlOptimizer
* RMSPropOptimizer

# 观察结果

# 作业二：请进行不同的参数调整，例如minibatch 大小，learning_rate，已经迭代训练的次数，观察结果

In [14]:
#2. AdadeltaOptimizer

In [ ]:
#3. AdagradOptimizer

In [ ]:
#4. MomentumOptimizer

In [ ]:
#5. AdamOptimizer

In [ ]:
#6. FtrlOptimizer

In [ ]:
#7. RMSPropOptimizer